# Dependencias

In [1]:
import numpy as np 
import pandas as pd
import nltk

# Load Data

In [2]:
data = pd.read_csv('./mails_clean.csv')
print(data.shape)

(42638, 10)


In [3]:
# tmp
#data = data.head(100)

In [4]:
data=data.drop(columns=['Unnamed: 0','X-cc','X-bcc'])
data=data.dropna(axis=0,subset=['body'],how='any')
print(data.shape)
data.head()

(42454, 7)


,file,Message-ID,Date,From,To,Subject,body
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,{'phillip.allen@enron.com'},{'tim.belden@enron.com'},NaN,Here is our forecast
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,{'phillip.allen@enron.com'},{'john.lavorato@enron.com'},Re:,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,{'phillip.allen@enron.com'},{'leah.arsdall@enron.com'},Re: test,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,{'phillip.allen@enron.com'},{'randall.gay@enron.com'},NaN,"Randy, Can you send me a schedule of the sal..."
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,{'phillip.allen@enron.com'},{'greg.piper@enron.com'},Re: Hello,Let's shoot for Tuesday at 11:45.


# Pre-Procesamiento

In [5]:
import re
def clean_from_mails(message):
  mail_addresses = re.findall(r'\(?[A-Za-z0-9_\-\.]+@[A-Za-zb0-9_\-\.]+\.com\)?',message)
  if mail_addresses != []:
    for mail_address in mail_addresses:
      message=message.replace(mail_address,'').strip()
  return message

In [6]:
# Eliminar emails del body
data['body_clean']=data['body'].map(clean_from_mails)

In [7]:
# Eliminar palabras con digitos y signos de puntuacion
data['body_clean']=data['body_clean'].map(lambda x: ' '.join(re.sub(r'[^a-zA-Z]',' ',x).split()))

In [8]:
# Pasar todo a minusculas
data['body_clean']=data['body_clean'].map(lambda x: x.lower())

## Stopwords

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kref/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
my_stopwords = stopwords.words('english')
# Excluir palabras de una sola letra
for i in range(ord('a'),ord('z')+1):
  my_stopwords.append(chr(i))
# Añadir would
my_stopwords = my_stopwords + ['also','please','would']
print(f'Stopwords final ({len(my_stopwords)}):')
print(my_stopwords)

Stopwords final (208):
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 

In [11]:
def clean_msg_stopwords(message_body):
    words = message_body.split()
    words = [w for w in words if w not in set(my_stopwords)]
    return ' '.join(words)

In [12]:
# Eliminar stopwords
data['body_clean']=data.body_clean.apply(clean_msg_stopwords)

In [13]:
data=data.dropna(axis=0,subset=['body_clean'],how='any')
print(data.shape)

(42454, 8)


## Tokenizacion

In [14]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/kref/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
word_vector = []
for doc in data['body_clean']:
  word_vector.append(doc.split())
my_vocabulary = [item for sublist in word_vector for item in sublist]

In [16]:
word_vector[4]

['let', 'shoot', 'tuesday']

In [17]:
print(f'El vocabulario consta de {len(my_vocabulary)} elementos y {len(set(my_vocabulary))} palabras diferentes.')

El vocabulario consta de 4047756 elementos y 44475 palabras diferentes.


## Lematizacion

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/kref/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

def lemmatize_body(message_body):
  return ' '.join(lemma.lemmatize(word) for word in message_body.split())

In [20]:
lemma_vector = []
for doc in data['body_clean']:
  lemma_vector.append(lemmatize_body(doc).split())
my_lema_voc = [item for sublist in lemma_vector for item in sublist]

In [21]:
print(f'El vocabulario de lemas consta de {len(my_lema_voc)} elementos y {len(set(my_lema_voc))} palabras diferentes.')

El vocabulario de lemas consta de 4047756 elementos y 40903 palabras diferentes.


In [22]:
data['body_lemma'] = data['body_clean'].map(lemmatize_body)

In [23]:
data['body_lemma'][4]

'let shoot tuesday'

# Vectorizacion

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
word_vector = []
for doc in data['body_lemma']:
  word_vector.append(doc.split())
my_vocabulary = [item for sublist in word_vector for item in sublist]

In [26]:
word_vector[4]

['let', 'shoot', 'tuesday']

In [27]:
data['body_lemma'][4]

'let shoot tuesday'

# Similaridad de Documentos

In [28]:
import gensim

In [29]:
dictionary = gensim.corpora.Dictionary(word_vector)

In [30]:
dictionary[78]

'tuesday'

In [31]:
print(dictionary.token2id["let"])

76


In [32]:
print(dictionary.token2id["shoot"])

77


In [33]:
print(dictionary.token2id["tuesday"])

78


In [34]:
print("Number of words in dictionary:",len(dictionary))

Number of words in dictionary: 40903


In [35]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in word_vector]

In [36]:
corpus[4]

[(76, 1), (77, 1), (78, 1)]

In [37]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=42454, num_nnz=2761697)


In [38]:
tf_idf[corpus[4]]

[(76, 0.22139430207687802), (77, 0.8676738950948895), (78, 0.4451141143333412)]

## Analisis de Clusters

In [46]:
from sklearn.decomposition import PCA

In [45]:
tf_idf[corpus]

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 6
clf = KMeans(n_clusters=n_clusters, max_iter=100, init='k-means++', n_init=1)
labels = clf.fit_predict(tf_idf[corpus])

## Procesamiento Similaridad

In [39]:
sims = gensim.similarities.Similarity('./',tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

/home/kref/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Similarity index with 42454 documents in 1 shards (stored under ./)
<class 'gensim.similarities.docsim.Similarity'>


## Query

In [40]:
def make_query_tf_idf(query):
    query_doc = [w.lower() for w in word_tokenize(query)]
    query_doc_bow = dictionary.doc2bow(query_doc)
    return tf_idf[query_doc_bow]

In [41]:
def get_n_most_similar(mail_id, n):
  query = data['body_lemma'][mail_id]
  print('------------------------ [ Query ] ------------------------')
  print(data['body'][mail_id])
  ranking = sims[make_query_tf_idf(query)]
  sim_w_index = list(enumerate(ranking))
  x = [t[0] for t in list(reversed(sorted(sim_w_index,key=lambda x: x[1])))]
  print()
  print('---------- [ Most similar emails (index, score)] ----------')
  print(list(reversed(sorted(sim_w_index,key=lambda x: x[1])))[:n])
  print()
  print('------------------------ [ emails] ------------------------')
  for ind in x[:n]:
    print('From: {}'.format(data['From'][ind]))
    print('To: {}'.format(data['To'][ind]))
    print(data['body'][ind])
    print('---------------------------------------------------------')

In [42]:
data['body_lemma'][45]

'lucy want accurate rent roll soon possible faxed copy file fill computer write correct amount input'

In [43]:
get_n_most_similar(45,5)

------------------------ [ Query ] ------------------------
Lucy,  I want to have an accurate rent roll as soon as possible. I faxed you a copy  of this file.  You can fill in on the computer or just write in the correct  amounts and I will input.

---------- [ Most similar emails (index, score)] ----------
[(2091, 1.0), (1187, 1.0), (616, 1.0), (45, 1.0), (1593, 0.39354533)]

------------------------ [ emails] ------------------------
From: {'phillip.allen@enron.com'}
To: {'jonathan.mckay@enron.com'}
John,  Here is our North of Stanfield forecast for Jan.   Supply    Jan '01   Dec '00   Jan '00   Sumas   900   910   815  Jackson Pr.  125    33   223  Roosevelt  300   298   333    Total Supply  1325   1241   1371  Demand  North of Chehalis 675   665   665  South of Chehalis 650   575   706   Total Demand  1325   1240   1371  Roosevelt capacity is 495.  Let me know how your forecast differs.   Phillip
---------------------------------------------------------
From: {'phillip.allen@enron.